# Text classification with RNNs
## Preamble: installing and importing packages

In [5]:
try:
    import datasets
except ModuleNotFoundError:
    !pip install datasets
    import datasets

In [6]:
try:
    from unidecode import unidecode
except ModuleNotFoundError:
    !pip install unidecode
    from unidecode import unidecode

In [7]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

## Load training dataset

We are going to work with a [ dataset that contains 58k carefully curated Reddit comments labeled for 27 emotions](https://www.tensorflow.org/datasets/catalog/goemotions). 
This dataset can be retreived using the [`datasets` library from the catalog of tensorflow ](https://huggingface.co/docs/datasets/index).

The next cells load some information on the dataset:

In [8]:
DATA_HANDLE = "geo_emotions"

In [9]:
from datasets import load_dataset_builder
ds_builder = load_dataset_builder(DATA_HANDLE)

FileNotFoundError: Couldn't find a dataset script at c:\Users\cham\Desktop\5A\predictive modeling and unsupervised clustering\ML-Supervised-learning\geo_emotions\geo_emotions.py or any data file in the same directory. Couldn't find 'geo_emotions' on the Hugging Face Hub either: FileNotFoundError: Dataset 'geo_emotions' doesn't exist on the Hub. If the repo is private, make sure you are authenticated with `use_auth_token=True` after logging in with `huggingface-cli login`.

In [ ]:
ds_builder.info.description

NameError: name 'ds_builder' is not defined

Each element in the dataset has two features: the review text itself, and the associated label:

In [ ]:
ds_builder.info.features

Now we are going to load the training data:

In [ ]:
from datasets import load_dataset

train_ds = load_dataset(DATA_HANDLE, split="train")

As seen in `ds_builder.info.features`, each data sample has two fields: the `review` text and the `label` string. Here is the review text for one particular sample

In [ ]:
train_ds[10]['review']

### Normalizing characters
Some of the tools we'll be using later cannot flawlessly handle all unicode characters. To avoid problems, we will normalize all characters to their closest ASCII equivalent using the function `unidecode` (imported from [`unidecode` package](https://pypi.org/project/Unidecode/)).

The function basically replaces all characters bearing [diacritic signs](https://en.wikipedia.org/wiki/Diacritic) with their corresponding plain character, as well as any symbols with close ASCII equivalents. The result is a text with no accents, cedillas, no € symbol, etc.

In [ ]:
unidecode(train_ds[10]['review'])